In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from functions import custom_cv_eval, get_best_features, get_ev_from_df, get_bet_ev
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
np.set_printoptions(suppress=True)  #Suppress Scientific Notation

In [2]:
df=pd.read_csv("data/owl-with-odds.csv")
df_event=pd.read_csv("data/upcoming-with-odds.csv")

In [3]:
df.shape

(683, 58)

In [4]:
df_event.shape

(12, 58)

In [5]:
subset = ['t1_odds', 't2_odds']
df.dropna(subset=subset ,inplace=True)

df['t1_odds'] = pd.to_numeric(df['t1_odds'], errors='coerce')
df['t2_odds'] = pd.to_numeric(df['t2_odds'], errors='coerce')
df.dropna(subset=subset ,inplace=True)

In [6]:
subset = ['t1_odds', 't2_odds']
df_event.dropna(subset=subset ,inplace=True)

df_event['t1_odds'] = pd.to_numeric(df_event['t1_odds'], errors='coerce')
df_event['t2_odds'] = pd.to_numeric(df_event['t2_odds'], errors='coerce')
df_event.dropna(subset=subset ,inplace=True)

In [7]:
df_train=df.copy()
df_test = df_event.copy()

In [8]:
#inclusive features
features = ['team_one', 'team_two', 'corona_virus_isolation', 't1_wins_season',
       't1_losses_season', 't2_wins_season', 't2_losses_season',
       't1_matches_season', 't2_matches_season', 't1_win_percent_season',
       't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime',
       't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime',
       't2_matches_alltime', 't1_win_percent_alltime',
       't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3',
       't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3',
       't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5',
       't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5',
       't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10',
       't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10',
       't2_win_percent_last_10', 
        't1_wins_vs_t2', 't1_losses_vs_t2',
       't1_matches_vs_t2', 't1_odds', 't2_odds', 'winner_label']

In [9]:
df_test_filtered = df_test[features].copy()
df_train_filtered = df_train[features].copy()
df_test_filtered.dropna(inplace=True)
df_train_filtered.dropna(inplace=True)

df_test_team_names = df_test_filtered[['team_one', 'team_two']]
df_train_team_names = df_train_filtered[['team_one', 'team_two']]

In [10]:
model_5 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')
features_5 = ['t1_wins_last_3', 't2_matches_alltime', 'winner_label', 't1_odds', 't2_odds']
df_5 = df_train_filtered[features_5]

In [11]:
display(df_test_team_names)

,team_one,team_two
0,Seoul Dynasty,Chengdu Hunters
1,London Spitfire,Shanghai Dragons
2,New York Excelsior,Hangzhou Spark
3,Vancouver Titans,Houston Outlaws
4,Florida Mayhem,Washington Justice
5,Los Angeles Gladiators,Boston Uprising
6,San Francisco Shock,Atlanta Reign
7,Guangzhou Charge,Chengdu Hunters
8,Hangzhou Spark,London Spitfire
9,Dallas Fuel,Paris Eternal


In [12]:
X=df_train_filtered[features_5].copy()
X_test=df_test_filtered[features_5].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_5.fit(X, y)
probs=model_5.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
final_probs_df = pd.concat([ev_prepped_df, df_test_team_names.reset_index()], axis=1)
#display(ev_prepped_df)
#get_ev_from_df(ev_prepped_df, True, min_ev=99)
#probs_5 = probs

ev_prepped_df.shape

(12, 5)

In [13]:
display(final_probs_df)

,t1_odds,t2_odds,t1_prob,t2_prob,winner,index,team_one,team_two
0,-500,400,1.000000,0.000000,0,0,Seoul Dynasty,Chengdu Hunters
1,625,-950,0.000000,1.000000,0,1,London Spitfire,Shanghai Dragons
2,-410,330,0.200000,0.800000,0,2,New York Excelsior,Hangzhou Spark
3,445,-590,0.800000,0.200000,0,3,Vancouver Titans,Houston Outlaws
4,-1250,800,0.571429,0.428571,0,4,Florida Mayhem,Washington Justice
5,-1250,800,0.833333,0.166667,0,5,Los Angeles Gladiators,Boston Uprising
6,-510,405,1.000000,0.000000,0,6,San Francisco Shock,Atlanta Reign
7,-300,240,0.875000,0.125000,0,7,Guangzhou Charge,Chengdu Hunters
8,130,-160,0.000000,1.000000,0,8,Hangzhou Spark,London Spitfire
9,145,-175,1.000000,0.000000,0,9,Dallas Fuel,Paris Eternal


In [14]:
min_ev = 99

for index, row in final_probs_df.iterrows():
    team_one_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
    team_two_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
    if (team_one_ev >=99):
        print(f"BET: {row['team_one']} ({row['t1_odds']}) over {row['team_two']} ({row['t2_odds']}). EV: {team_one_ev}")
        
    if (team_two_ev >=99):
        print(f"BET: {row['team_two']} ({row['t2_odds']}) over {row['team_one']} ({row['t1_odds']}). EV: {team_two_ev}")

BET: Hangzhou Spark (330) over New York Excelsior (-410). EV: 244.0
BET: Vancouver Titans (445) over Houston Outlaws (-590). EV: 336.0
BET: Washington Justice (800) over Florida Mayhem (-1250). EV: 285.71428571428567
BET: Dallas Fuel (145) over Paris Eternal (-175). EV: 145.0
BET: Vancouver Titans (900) over Philadelphia Fusion (-1610). EV: 500.0
